In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sqlalchemy
import urllib.parse
pd.set_option('display.max_rows',None)

In [2]:
# Define your database connection variables
from sqlalchemy import create_engine
import pandas as pd

# Define your database credentials
dialect = "mysql"
driver = "pymysql"
username = "root"
password = "Root123"
host = "localhost"
db_name = "CustomerPurchaseBehaviour"

# Create the database URI
db_uri = f"{dialect}+{driver}://{username}:{password}@{host}/{db_name}"

# Create the database engine
my_engine = create_engine(db_uri)

# Define the SQL query
query = "SELECT * FROM CUSTOMERS"

# Execute the query and load the results into a DataFrame
with my_engine.connect() as db:
    customers = pd.read_sql(query, db)

# Display the DataFrame
print(customers)

     CustomerID                CustomerName  \
0           100                 Mary Chavez   
1           100             Victoria Larson   
2           102               Ronald Brandt   
3           104                Cheryl Downs   
4           104              Richard Joseph   
5           105              Andrew Esparza   
6           107             Brandon Pearson   
7           107              Christine Soto   
8           108                Chris Snyder   
9           108               Eddie Mueller   
10          108                Kayla Abbott   
11          108            Matthew Alvarado   
12          108               Ryan Reynolds   
13          109            Deborah Mitchell   
14          109           Mr. James Hoffman   
15          111            Kristen Martinez   
16          112                James Miller   
17          112              Megan Saunders   
18          114                Andrew Brown   
19          115             Colton Figueroa   
20          1

Converting SQL data to Dataframe

In [3]:
# Getting the products table to a dataframe
query = "SELECT * FROM PRODUCTS"
# Execute the query and load the results into a DataFrame
with my_engine.connect() as db:
    products = pd.read_sql(query, db)

# Display the DataFrame
print(products)

     ProductID      ProductName  ProductCategory
0         1001           Router      Electronics
1         1002           Camera      Electronics
2         1003  Electric Kettle  Home Appliances
3         1004       Smartwatch      Electronics
4         1005          Toaster  Home Appliances
5         1006       Television      Electronics
6         1007        Microwave  Home Appliances
7         1008           Heater  Home Appliances
8         1009          Printer      Electronics
9         1010  Air Conditioner  Home Appliances
10        1011           Router      Electronics
11        1012           Laptop      Electronics
12        1013     Refrigerator  Home Appliances
13        1014       Television      Electronics
14        1015           Tablet      Electronics
15        1016           Tablet      Electronics
16        1017          Printer      Electronics
17        1018       Smartphone      Electronics
18        1019       Headphones      Electronics
19        1020      

In [4]:
# Getting the categories table to a dataframe
query = "SELECT * FROM ORDERDETAILS"
# Execute the query and load the results into a DataFrame
with my_engine.connect() as db:
    orderdetails = pd.read_sql(query, db)

# Display the DataFrame
print(orderdetails)

     TransactionID PurchaseDate  ProductID  CustomerID  \
0                1   2024-03-01       1001         887   
1                2   2024-01-26       1002         560   
2                3   2024-05-13       1003         701   
3                4   2023-09-21       1004         630   
4                5   2024-01-25       1005         631   
5                6   2023-08-30       1006         228   
6                7   2023-11-04       1007         694   
7                8   2024-04-20       1008         677   
8                9   2023-07-07       1009         371   
9               10   2024-05-03       1010         742   
10              11   2023-12-13       1011         801   
11              12   2023-10-28       1012         595   
12              13   2024-01-30       1013         981   
13              14   2023-08-02       1014         426   
14              15   2024-05-06       1015         397   
15              16   2023-12-24       1016         691   
16            

In [5]:
# Merging the datasets to get the full dataset, this will help to create graphs and analyze data
newdf = orderdetails.merge(customers, how='inner',on=["CustomerID","CustomerName"])
final_data=newdf.merge(products, how='inner',on="ProductID")
print(final_data)

     TransactionID PurchaseDate  ProductID  CustomerID  \
0                1   2024-03-01       1001         887   
1                2   2024-01-26       1002         560   
2                3   2024-05-13       1003         701   
3                4   2023-09-21       1004         630   
4                5   2024-01-25       1005         631   
5                6   2023-08-30       1006         228   
6                7   2023-11-04       1007         694   
7                8   2024-04-20       1008         677   
8                9   2023-07-07       1009         371   
9               10   2024-05-03       1010         742   
10              11   2023-12-13       1011         801   
11              12   2023-10-28       1012         595   
12              13   2024-01-30       1013         981   
13              14   2023-08-02       1014         426   
14              15   2024-05-06       1015         397   
15              16   2023-12-24       1016         691   
16            

In [6]:
# Changing the date column to date-
final_data['PurchaseDate']= pd.to_datetime(final_data['PurchaseDate'])

In [7]:
# Checking the conversion
print(final_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   TransactionID     1000 non-null   int64         
 1   PurchaseDate      1000 non-null   datetime64[ns]
 2   ProductID         1000 non-null   int64         
 3   CustomerID        1000 non-null   int64         
 4   CustomerName      1000 non-null   object        
 5   PurchaseQuantity  1000 non-null   int64         
 6   PurchasePrice     1000 non-null   float64       
 7   Country           1000 non-null   object        
 8   ProductName       1000 non-null   object        
 9   ProductCategory   1000 non-null   object        
dtypes: datetime64[ns](1), float64(1), int64(4), object(4)
memory usage: 78.3+ KB
None


DATA ANALYSIS

In [8]:
# How many transactions are there-
len(final_data["TransactionID"])

1000

In [9]:
# What is the total Revenue
total_revenue = (final_data["PurchasePrice"] * final_data["PurchaseQuantity"]).sum()
print(total_revenue)

1485760.55


In [10]:
# What is the total Purchase Price
total_purchase_price = final_data["PurchasePrice"].sum()
print(total_purchase_price)

489274.72


In [11]:
# What is the average purchase price
print(round(final_data["PurchasePrice"].mean(),2))

489.27


In [12]:
# Category wise total Purchase Price
final_data.groupby(['ProductCategory'], sort=True)['PurchasePrice'].sum()

ProductCategory
Electronics        248194.56
Home Appliances    241080.16
Name: PurchasePrice, dtype: float64

In [13]:
# Category wise total quantity sold:
final_data.groupby(['ProductCategory'],sort=True)['PurchaseQuantity'].sum()

ProductCategory
Electronics        1480
Home Appliances    1573
Name: PurchaseQuantity, dtype: int64

In [14]:
# Product wise total Purchase Price:
new_df = final_data.groupby(['ProductName'], sort=True)['PurchasePrice'].sum().reset_index()
new_df = new_df.sort_values(by = 'PurchasePrice', ascending=False)
print(new_df)

        ProductName  PurchasePrice
8         Microwave       30970.75
14       Smartwatch       29407.68
17          Toaster       29116.03
6            Heater       28828.67
5        Headphones       27748.78
10     Refrigerator       27730.95
9           Printer       27360.29
16       Television       25188.60
2            Camera       24985.70
0   Air Conditioner       24812.05
7            Laptop       24493.14
12       Smart Bulb       22575.66
4   Electric Kettle       22566.28
13       Smartphone       22508.32
15           Tablet       22028.51
18   Vacuum Cleaner       21921.68
11           Router       21897.88
19  Washing Machine       19987.46
3      Coffee Maker       19517.16
1           Blender       15629.13


In [15]:
# Product wise total quantity:

new_df = final_data.groupby(['ProductName'], sort=True)['PurchaseQuantity'].sum().reset_index()
new_df = new_df.sort_values(by = 'PurchaseQuantity', ascending=False)
new_df.rename(columns={'PurchaseQuantity':'TotalQuantity'},inplace= True)
new_df

,ProductName,TotalQuantity
17,Toaster,191
6,Heater,183
8,Microwave,180
10,Refrigerator,167
0,Air Conditioner,165
9,Printer,163
16,Television,161
14,Smartwatch,159
7,Laptop,154
12,Smart Bulb,152


In [16]:
# Top 10 products by Quantity:
final_data.groupby(['ProductName'])['PurchaseQuantity'].sum().nlargest(10)

ProductName
Toaster            191
Heater             183
Microwave          180
Refrigerator       167
Air Conditioner    165
Printer            163
Television         161
Smartwatch         159
Laptop             154
Smart Bulb         152
Name: PurchaseQuantity, dtype: int64

In [17]:
# Top 10 customers in terms of purchase price
final_data.groupby(['CustomerName'])['PurchasePrice'].sum().nlargest(10)

CustomerName
Brittany Miller      1324.45
Charles Hernandez    1289.25
James Miller         1080.46
Christopher Hill     1034.04
Jillian Baker         999.98
Patricia Taylor       999.44
Mikayla Rios          998.17
Matthew Espinoza      997.45
Jamie Montoya         996.01
Charles Clark         995.97
Name: PurchasePrice, dtype: float64

In [18]:
final_data.groupby(['CustomerName','ProductName','ProductCategory'])['PurchasePrice'].sum().nlargest(10)

CustomerName      ProductName     ProductCategory
Jillian Baker     Vacuum Cleaner  Home Appliances    999.98
Patricia Taylor   Smartphone      Electronics        999.44
Mikayla Rios      Camera          Electronics        998.17
Matthew Espinoza  Television      Electronics        997.45
Jamie Montoya     Headphones      Electronics        996.01
Charles Clark     Smartphone      Electronics        995.97
Antonio Hughes    Vacuum Cleaner  Home Appliances    994.58
Carlos Smith      Headphones      Electronics        993.55
Steven Rivera     Refrigerator    Home Appliances    992.33
Casey Johnson     Television      Electronics        991.08
Name: PurchasePrice, dtype: float64

In [19]:
# Country wise Purchase Price
new_df = final_data.groupby(['Country'], sort=True)['PurchasePrice'].sum().reset_index()
new_df = new_df.sort_values(by = 'PurchasePrice', ascending=False)
new_df.rename(columns={'PurchasePrice':'TotalPurchasePrice'},inplace= True)
new_df

,Country,TotalPurchasePrice
162,Palau,5584.26
200,Sudan,5496.35
89,Guyana,5410.66
23,Bhutan,5317.89
173,Romania,4944.19
122,Luxembourg,4866.90
18,Barbados,4824.66
66,Falkland Islands (Malvinas),4482.61
111,Korea,4479.61
179,Saint Lucia,4324.02


In [20]:
# Top 10 countries by Purchase Price:
final_data.groupby(['Country'])['PurchasePrice'].sum().nlargest(10)

Country
Palau                          5584.26
Sudan                          5496.35
Guyana                         5410.66
Bhutan                         5317.89
Romania                        4944.19
Luxembourg                     4866.90
Barbados                       4824.66
Falkland Islands (Malvinas)    4482.61
Korea                          4479.61
Saint Lucia                    4324.02
Name: PurchasePrice, dtype: float64

In [21]:
# Country wise quantity sold:
new_df = final_data.groupby(['Country'], sort=True)['PurchaseQuantity'].sum().reset_index()
new_df = new_df.sort_values(by = 'PurchaseQuantity', ascending=False)
new_df.rename(columns={'PurchaseQuantity':'TotalQuantity'},inplace= True)
new_df

,Country,TotalQuantity
166,Philippines,34
199,Sri Lanka,31
18,Barbados,30
173,Romania,30
162,Palau,29
60,Egypt,28
63,Eritrea,28
200,Sudan,28
26,Botswana,27
122,Luxembourg,26


In [22]:
# What is the total quantity sold:
print(final_data["PurchaseQuantity"].sum())

3053


In [23]:
# Year wise number of customers:
new_df = final_data.groupby([final_data.PurchaseDate.dt.year], sort=True)['CustomerID'].count().reset_index()
new_df = new_df.sort_values(by = 'CustomerID', ascending=False)
new_df.rename(columns={'CustomerID':'Customer_Count'},inplace= True)
new_df['Percentage']=(new_df['Customer_Count'] /
                  new_df['Customer_Count'].sum()) * 100
new_df

,PurchaseDate,Customer_Count,Percentage
0,2023,543,54.3
1,2024,457,45.7


In [24]:
# Year wise Total Purchase Price:
new_df = final_data.groupby([final_data.PurchaseDate.dt.year], sort=True)['PurchasePrice'].sum().reset_index()
new_df = new_df.sort_values(by = 'PurchasePrice', ascending=False)
new_df.rename(columns={'PurchasePrice':'Total_Purchase_Price'},inplace= True)
new_df.rename(columns={'PurchaseDate':'Year'},inplace= True)
new_df['Growth_Rate'] = new_df['Total_Purchase_Price'].pct_change(periods=1) * 100
new_df

,Year,Total_Purchase_Price,Growth_Rate
0,2023,262346.07,NaN
1,2024,226928.65,-13.500267


In [25]:
# month wise Total Purchase Price:
new_df = final_data.groupby([final_data.PurchaseDate.dt.month ], sort=True)['PurchasePrice'].sum().reset_index()
new_df = new_df.sort_values(by = 'PurchasePrice', ascending=False)
new_df.rename(columns={'PurchasePrice':'Total_Purchase_Price'},inplace= True)
new_df.rename(columns={'PurchaseDate':'Month'},inplace= True)
new_df

,Month,Total_Purchase_Price
11,12,51910.42
6,7,47956.69
8,9,46764.80
0,1,43575.14
4,5,42799.97
1,2,41393.77
10,11,39839.29
9,10,38368.97
2,3,37442.66
3,4,34165.64


In [26]:
# Quarter wise Total Purchase Price:
new_df = final_data.groupby([final_data.PurchaseDate.dt.quarter ], sort=True)['PurchasePrice'].sum().reset_index()
new_df = new_df.sort_values(by = 'PurchasePrice', ascending=False)
new_df.rename(columns={'PurchasePrice':'Total_Purchase_Price'},inplace= True)
new_df.rename(columns={'PurchaseDate':'Quarter'},inplace= True)
new_df

,Quarter,Total_Purchase_Price
3,4,130118.68
2,3,128214.95
0,1,122411.57
1,2,108529.52


In [27]:
# Creating a copy of the original data and extracting year,month and quarter of purchase date into new columns
final_data_copy=final_data
final_data_copy['Year']=final_data_copy['PurchaseDate'].dt.year
final_data_copy['Month']=final_data_copy['PurchaseDate'].dt.month
final_data_copy['Quarter']=final_data_copy['PurchaseDate'].dt.quarter

In [28]:
# Year wise month wise Total Purchase Price with growth
new_df = final_data_copy.groupby(['Year','Month'], sort=True)['PurchasePrice'].sum().reset_index()
#new_df = new_df.sort_values(by = 'Purchase_Price', ascending=False)
new_df.rename(columns={'PurchasePrice':'Total_Purchase_Price'},inplace= True)
new_df['Growth_Rate'] = new_df['Total_Purchase_Price'].pct_change(periods=1) * 100
new_df

,Year,Month,Total_Purchase_Price,Growth_Rate
0,2023,6,4012.44,NaN
1,2023,7,47956.69,1095.200177
2,2023,8,33493.46,-30.158941
3,2023,9,46764.80,39.623676
4,2023,10,38368.97,-17.953311
5,2023,11,39839.29,3.832055
6,2023,12,51910.42,30.299561
7,2024,1,43575.14,-16.057046
8,2024,2,41393.77,-5.005997
9,2024,3,37442.66,-9.545180


In [29]:
# Year wise month wise Total Purchase Price
new_df = final_data_copy.groupby(['Year','Month'], sort=True)['PurchasePrice'].sum().reset_index()
new_df = new_df.sort_values(by = 'PurchasePrice', ascending=False)
new_df.rename(columns={'PurchasePrice':'Total_Purchase_Price'},inplace= True)
new_df

,Year,Month,Total_Purchase_Price
6,2023,12,51910.42
1,2023,7,47956.69
3,2023,9,46764.80
7,2024,1,43575.14
11,2024,5,42799.97
8,2024,2,41393.77
5,2023,11,39839.29
4,2023,10,38368.97
9,2024,3,37442.66
10,2024,4,34165.64


In [30]:
#Year wise quarter wise total Purchase Price:
new_df = final_data_copy.groupby(['Year','Quarter'], sort=True)['PurchasePrice'].sum().reset_index()
new_df = new_df.sort_values(by = 'PurchasePrice', ascending=False)
new_df.rename(columns={'PurchasePrice':'Total_Purchase_Price'},inplace= True)
new_df

,Year,Quarter,Total_Purchase_Price
2,2023,4,130118.68
1,2023,3,128214.95
3,2024,1,122411.57
4,2024,2,104517.08
0,2023,2,4012.44


In [31]:
# Year wise quarter wise growth:
#Year wise quarter wise total Purchase Price:
new_df = final_data_copy.groupby(['Year','Quarter'], sort=True)['PurchasePrice'].sum().reset_index()
new_df.rename(columns={'PurchasePrice':'Total_Purchase_Price'},inplace= True)
new_df['Growth_Rate'] = new_df['Total_Purchase_Price'].pct_change(periods=1) * 100
new_df

,Year,Quarter,Total_Purchase_Price,Growth_Rate
0,2023,2,4012.44,NaN
1,2023,3,128214.95,3095.435944
2,2023,4,130118.68,1.484796
3,2024,1,122411.57,-5.923139
4,2024,2,104517.08,-14.618300


In [32]:
# Year wise total quantity:
new_df = final_data.groupby([final_data.PurchaseDate.dt.year], sort=True)['PurchaseQuantity'].sum().reset_index()
new_df = new_df.sort_values(by = 'PurchaseQuantity', ascending=False)
new_df.rename(columns={'PurchaseDate':'Year'},inplace= True)
new_df

,Year,PurchaseQuantity
0,2023,1651
1,2024,1402


In [33]:
# Year wise month wise total quantity:
new_df = final_data_copy.groupby(['Year','Month'], sort=True)['PurchaseQuantity'].sum().reset_index()
new_df = new_df.sort_values(by = 'PurchaseQuantity', ascending=False)
new_df

,Year,Month,PurchaseQuantity
6,2023,12,295
5,2023,11,293
7,2024,1,292
3,2023,9,287
8,2024,2,272
1,2023,7,269
11,2024,5,268
4,2023,10,243
2,2023,8,236
9,2024,3,208


In [34]:
# Year wise quarter wise total quantity:
new_df = final_data_copy.groupby(['Year','Quarter'], sort=True)['PurchaseQuantity'].sum().reset_index()
new_df = new_df.sort_values(by = 'PurchaseQuantity', ascending=False)
new_df

,Year,Quarter,PurchaseQuantity
2,2023,4,831
1,2023,3,792
3,2024,1,772
4,2024,2,630
0,2023,2,28


In [35]:
# Country wise customer count-
# Year wise number of customers:
new_df = final_data.groupby(["Country"], sort=True)['CustomerID'].count().reset_index()
new_df = new_df.sort_values(by = 'CustomerID', ascending=False)
new_df.rename(columns={'CustomerID':'Customer_Count'},inplace= True)
new_df

,Country,Customer_Count
200,Sudan,11
162,Palau,10
166,Philippines,9
173,Romania,9
26,Botswana,9
199,Sri Lanka,9
111,Korea,8
144,Mozambique,8
122,Luxembourg,8
23,Bhutan,8


In [36]:
# Which product sells the most in terms of quantity in each month?
df=final_data_copy.groupby(['Month','ProductName']).agg(total_quantity=("PurchaseQuantity","sum"))
df=df.reset_index()
idx = df.groupby('Month')['total_quantity'].idxmax()
df = df.loc[idx]
df

,Month,ProductName,total_quantity
11,1,Router,30
27,2,Laptop,25
57,3,Vacuum Cleaner,25
66,4,Laptop,20
86,5,Microwave,24
114,6,Toaster,22
127,7,Refrigerator,32
154,8,Toaster,26
173,9,Television,28
177,10,Air Conditioner,28


In [37]:
# Top 5 products for each category in terms of quantity:
df=final_data_copy.groupby(['ProductCategory','ProductName']).agg(total_quantity=("PurchaseQuantity","sum"))
df=df.reset_index()
df.loc[df.groupby('ProductCategory')['total_quantity'].nlargest(5).reset_index(0).index]

,ProductCategory,ProductName,total_quantity
3,Electronics,Printer,163
9,Electronics,Television,161
7,Electronics,Smartwatch,159
2,Electronics,Laptop,154
5,Electronics,Smart Bulb,152
17,Home Appliances,Toaster,191
14,Home Appliances,Heater,183
15,Home Appliances,Microwave,180
16,Home Appliances,Refrigerator,167
10,Home Appliances,Air Conditioner,165


In [38]:
# Which product gives the most revenue each year?
df=final_data_copy.groupby(['Year','ProductCategory','ProductName']).agg(total_price=("PurchasePrice","sum"))
df=df.reset_index()
idx = df.groupby(['Year','ProductCategory'])['total_price'].idxmax()
df = df.loc[idx]
df

,Year,ProductCategory,ProductName,total_price
7,2023,Electronics,Smartwatch,17897.15
10,2023,Home Appliances,Air Conditioner,18395.71
23,2024,Electronics,Printer,15132.63
35,2024,Home Appliances,Microwave,15799.85


SUMMARY REPORT

1. Data Period:

The dataset encompasses transactions from the years 2023 and 2024.

2. Total Transactions:

The dataset includes a total of 1,000 transactions.

3. Total Revenue:

The total revenue generated from these transactions is $1485760.55. 

4. Total Products Sold:

The dataset records the sale of 3,053 products.

5. Categories and Total Purchase Price:

There are two product categories represented in the dataset:
Electronics: Having Total Purchase Price of $248,194.56.
Home Appliances: Having Total Purchase Price of $241,080.16.
The Electronics category has a slightly higher Purchase Price compared to Home Appliances.

6. Category-wise Product Quantities:

The total quantities sold for each category are:

Home Appliances: 1,573 products.
Electronics: 1,480 products.

7. Top Total Purchase Price Product:

The product with the highest total Purchase Price is the Microwave, which has generated $30,970.75.

8. Top Quantity Sold Product:

Toasters are the most sold product, with a total of 191 units sold.

9. Top 10 Customers by Purchase Value:

The top 10 customers, assuming each customer can make up to 2 transactions, are:
Brittany Miller  
Charles Hernandez 
James Miller 
Jillian Baker 
Christopher Hill 
Patricia Taylor 
Mikayla Rios 
Matthew Espinoza 
Jamie Montoya 
Charles Clark 

10. Top 10 Countries by Purchase Value:

The top 10 countries are: Palau(5584.26), Sudan(5496.35), Guyana(5410.66), Bhutan(5317.89), Romania(4944.19), Luxembourg(4866.90), (Barbados)4824.66, Falkland Islands(Malvinas)(4482.61), 
Korea(4479.61), Saint Lucia(4324.02)
